# Testing our LRU Cache for Arbitrage Optimization

The biggest bottleneck in our code is finding the optimal arbitrage for our trades. One way we have optimized this is by implementing an LRU cache on the `cycle.optimize()` method. Here, we test that it works.

In [1]:
import time
from src.sim.scenario import Scenario
from src.trades.cycle import _optimize_mem

In [2]:
# Generate markets and prices
scenario = Scenario("baseline")
sample = scenario.pricepaths[0]
scenario.prepare_for_run()  # Set block timestamps
scenario.prepare_for_trades(sample)  # Set External Market Price

[INFO][01:05:48][src.sim.scenario]-890948: Using 658080 1Inch quotes.
[INFO][01:05:48][src.sim.scenario]-890948: Fetching sim_market from subgraph.
[INFO][01:05:51][src.utils.poolgraph]-890948: Found 20 valid cycles of length 3.


In [3]:
# Unpack
cycles = scenario.cycles
cycle = cycles[0]
trade = cycle.trades[0]
decimals = trade.pool.coin_decimals[trade.i]
address = trade.pool.coin_addresses[trade.i].lower()

# Set xatol
xatol = int(10**decimals / sample.prices_usd[address])

In [4]:
# Check single cycle caching
start = time.time()
cycle.optimize(xatol)
end = time.time()
print(f"Before caching: {end - start}")
start = time.time()
cycle.optimize(xatol)
end = time.time()
print(f"After caching: {end - start}")
_optimize_mem.cache_info()  # Ensure there was one hit and one miss!

Before caching: 0.0062906742095947266
After caching: 6.556510925292969e-05


CacheInfo(hits=1, misses=1, maxsize=1000, currsize=1)

In [5]:
cycle.execute()
cycle.optimize(xatol)
_optimize_mem.cache_info()  # Ensure the cache wasn't hit this time

[INFO][01:05:51][src.trades.cycle]-890948: Executing cycle Cycle(Trades: [Swap(pool=Curve.fi Factory Plain Pool: crvUSD/TUSD, in=TUSD, out=crvUSD, amt=363672183276034688), Swap(pool=Curve.fi Factory Plain Pool: crvUSD/USDP, in=crvUSD, out=USDP, amt=364486001351475391), Swap(pool=External Market (TUSD, USDP), in=Pax Dollar, out=TrueUSD, amt=362907044194404510)], Expected Profit: -0.000280738285457792).
[INFO][01:05:51][src.trades.cycle]-890948: Executing trade Swap(pool=Curve.fi Factory Plain Pool: crvUSD/TUSD, in=TUSD, out=crvUSD, amt=363672183276034688).
[INFO][01:05:51][src.trades.cycle]-890948: Executing trade Swap(pool=Curve.fi Factory Plain Pool: crvUSD/USDP, in=crvUSD, out=USDP, amt=364486001351475391).
[INFO][01:05:51][src.trades.cycle]-890948: Executing trade Swap(pool=External Market (TUSD, USDP), in=Pax Dollar, out=TrueUSD, amt=362907044194404510).


CacheInfo(hits=1, misses=2, maxsize=1000, currsize=2)

In [6]:
cycle.optimize(xatol)
_optimize_mem.cache_info()  # Ensure the cache was hit this time

CacheInfo(hits=2, misses=2, maxsize=1000, currsize=2)

In [7]:
sample = scenario.pricepaths[1]
xatol = int(10**decimals / sample.prices_usd[address])
scenario.prepare_for_trades(sample)  # Update External Market Price
cycle.optimize(xatol)
_optimize_mem.cache_info()  # Ensure the cache wasn't hit this time because prices changed

CacheInfo(hits=2, misses=3, maxsize=1000, currsize=3)

In [8]:
# Nice!